# Chinese Poetry Genarator - V1 -With RNN

In [1]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [3]:
import jieba as jb
import numpy as np
from rnn_utils import *
from nlp_helper import load_char_embeddings, load_corpus

In [4]:
char_to_index, index_to_char, index_to_vec = load_char_embeddings("../../data/embedding/sogou/char_embedding_plus_punctuation")

In [5]:
vocab_len = len(char_to_index) + 2
emb_dim = index_to_vec['0'].shape
print("vocab_len = ", vocab_len, "emb_dim = ", emb_dim)

vocab_len =  6902 emb_dim =  (300,)


In [6]:
# embedding fitures are between -1.685101 1.671867
# append EOS and UNK to embedding matrixz
np.random.seed(ord('E'))
char_to_index['<EOS>'] = 6900
index_to_char['6900'] = '<EOS>'
index_to_vec['6900'] = np.random.rand(300,)

np.random.seed(ord('U'))
char_to_index['<UNK>'] = 6901
index_to_char['6901'] = '<UNK>'
index_to_vec['6901'] = np.random.rand(300,)

#define /n as EOS
char_to_index['\n'] = 6900

In [7]:
len(index_to_vec)

6902

In [35]:
tangshis = np.load("../../data/chinese-poetry-master/tangshi_v1_5yan.npy")

In [36]:
tangshis[0]

'秦川雄帝宅，函谷壮皇居。'

In [37]:
# dataset= tangshis
# index = 5
# batch_size = 4
# mini_batch = []
# Y = []
# for c in range(12):
#     char_batch = []
#     char_batch_ix = []
#     for i in range(batch_size): 
#         ix = char_to_index['<EOS>'] #初始化而已
#         try:
#             ix = char_to_index[dataset[index+i][c]]
#         except KeyError:
#             ix = char_to_index['<UNK>'] 
#         char_batch_ix.append(ix)
#         char_batch.append(index_to_vec[str(ix)])
#     Y.append(char_batch_ix)
#     mini_batch.append(char_batch)
# Y.append([char_to_index['<EOS>']]*4)
# Y

In [38]:
def save_params(parameters):
    np.save("saves/parameters_"+str(parameters['endpoint'])+'.npy', parameters)

In [8]:
def sample(parameters, char_to_ix, ix_to_char, ix_to_vec, seed, fixed_chars = None, padding = False ):
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((300,1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a,1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugxging and prevents entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 12):
       
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        
        a = np.tanh(np.dot( Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        
        # for grading purposes
        np.random.seed(counter+seed) 
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        if fixed_chars!=None and counter<len(fixed_chars):
            idx = char_to_ix[fixed_chars[counter]]
        else:
            while True:
                idx = np.random.choice(range(vocab_size),p = y.ravel())
                if not padding or idx != char_to_index["<EOS>"]:

                    break
            
        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        x = index_to_vec[str(idx)].reshape(300,1)
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [40]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]
    

In [76]:
def model(dataset,index_to_char, char_to_index, index_to_vec, num_iterations = 120000, n_a = 64,gen_samples = 5,parameters = None, batch_size = 64,learning_rate= 0.01):
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = 300, len(char_to_index)
    
    # Initialize parameters
    if not parameters:
        parameters = initialize_parameters(n_a, n_x, n_y)
        parameters['endpoint'] = 0
        parameters['time'] = []
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    # loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(dataset)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, batch_size))
    j = 0
    import time
    time_start=time.time()
    epoch_size = len(dataset)
    try:
    # Optimization loop
        for j in range(parameters['endpoint'],parameters['endpoint'] + num_iterations):

            ### START CODE HERE ###

            # Use the hint above to define one training example (X,Y) (≈ 2 lines)
            index = j*batch_size%len(dataset)
            
            
            

            X = []
            Y = []
            
            # 讲汉字转换为向量输入和索引输出，并处理未知字符。生成输入矩阵
            X.append(np.zeros((300,batch_size)))
            for c in range(12):
                char_batch = []
                char_batch_ix = []
                for i in range(batch_size): 
                    ix = char_to_index['<EOS>'] #初始化而已
                    try:
                        ix = char_to_index[dataset[index+i][c]]
                    except KeyError:
                        ix = char_to_index['<UNK>'] 
                        
                    char_batch_ix.append(ix)
                    char_batch.append(index_to_vec[str(ix)])
                    
                X.append(np.array(char_batch).T)
                Y.append(char_batch_ix)
            
            Y.append([char_to_index["<EOS>"]]*batch_size)

            

            # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
            # Choose a learning rate of 0.01
            loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = learning_rate)

            # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
            # loss = smooth(loss, curr_loss)

            # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
            
            print("进度:{}/{} epoches".format(j*batch_size,epoch_size), end="\r")
             
            if j*batch_size % 512 == 0:
                print("\n耗时",round(time.time()-time_start,3),"s")
                print('Iteration: %d, Loss: %7f' % (j*batch_size, loss) + '\n')

                # The number of dinosaur names to print
                seed = 0
                for s in range(gen_samples):
                    
                    # Sample indices and print them
                    sampled_indices = sample(parameters, char_to_index, index_to_char, index_to_vec, seed)
                    print_sample(sampled_indices, index_to_char)
                    print('')
                    seed += 1  # To get the same result for grading purposed, increment the seed by one. 

                print('\n')
    except KeyboardInterrupt:
        pass
    finally:
        
        parameters['endpoint'] = j
        time_end=time.time()
        parameters['time'].append(round(time_end-time_start,3))
        save_params(parameters)
         
    return parameters

In [10]:
 parameters= np.load("saves/parameters_360453.npy")[()]

In [93]:
# parameters['time'] 

In [94]:
parameters=model(tangshis, index_to_char, char_to_index, index_to_vec, num_iterations = 10000,batch_size=64,parameters=parameters, learning_rate=0.05)

进度:12189184/154162 epoches
耗时 0.674 s
Iteration: 12189184, Loss: 74.633679

毛马翊棱賨，高极得营亭。
定动没以为，忧贱报两年。
何□萧萧枝，复开草木深。
仙开不为此，蒲子不不知。
<EOS>


进度:12189696/154162 epoches
耗时 2.5 s
Iteration: 12189696, Loss: 74.560854

毛马狸樵憔，好梦几眼积。
定动余行在，耳臣即大名。
何□萧萧舍，临城终相传。
仙开不为此，慈许不不知。
<EOS>


进度:12190208/154162 epoches
耗时 4.222 s
Iteration: 12190208, Loss: 72.987428

形随稽昼狭，处合自逢诗。
定合如及将，缝茸转新天。
何□萧萧宅，何之玉影亩。
仙开不为此，慈许不不知。
<EOS>


进度:12190720/154162 epoches
耗时 5.921 s
Iteration: 12190720, Loss: 74.171253

形随骎烛贞，近客几铁闻。
定合如我是，陶朔便月多。
何□萧萧挥，春间柳柳回。
仙台多一去，遥见不从端。
<EOS>


进度:12191232/154162 epoches
耗时 7.655 s
Iteration: 12191232, Loss: 76.603394

形随悭丛悄，如照没相逢。
辟食起月将，诸贱遭其多。
何□萧萧识，调花遍舞脸。
仙台多一去，遥见不从营。
<EOS>


进度:12191744/154162 epoches
耗时 9.483 s
Iteration: 12191744, Loss: 75.838346

形委辄忽遣，几戏当逾婚。
定合如我是，陶杖自有多。
何□萧萧识，调花遍舞脚。
仙台多一去，囊深时将落。
<EOS>


进度:12192256/154162 epoches
耗时 11.18 s
Iteration: 12192256, Loss: 75.572068

形别绛鸾，虚天物同。<EOS>
辟云无人在，始愁问地三。
何□萧萧幸，春开献扇科。
仙台多一去，彼此及不守。
<EOS>


进度:12192768/154162 epoches
耗时 12.842 s
Iteratio

In [34]:
save_params(parameters)

In [15]:
# 生成 诗句
seed = 10
for name in range(5):
                
    # Sample indices and print them
    sampled_indices = sample(parameters, char_to_index, index_to_char, index_to_vec, seed, fixed_chars="", padding = False)
    print_sample(sampled_indices, index_to_char)
    seed += 1  # To get the same result for grading purposed, increment the seed by one. 
      
    print('\n')

扶看云水曲，数讶片寒涛。

近言披云里，云摧动丘峰。

好借经旅细，魑盖甚何劳，

凤霜龙城上，哀歌凌干清。

何时乱帝星。<EOS>

